# Investigating the Network

## Intro / Goal

## Setup

## Device Mapping

| IP                                    | Sven's Device | Maxi's Device | Fabi's Devices                         |
| ------------------------------------- | ------------- | ------------- | -------------------------------------- |
| 192.168.178.1                         | Router        |               |                                        |
| 192.168.178.24                        | Google Home   |               |                                        |
| 192.168.178.26                        | Chromecast    |               |                                        |
| 192.168.178.27                        | Google Home   |               |                                        |
| 192.168.178.29                        | Google Home   |               |                                        |
| 192.168.178.42                        | Google Home   |               |                                        |
| 192.168.178.43                        | SmartTV       |               |                                        |
| 192.168.178.44                        | Android Phone |               |                                        |
| 192.168.178.50                        | iPad          |               |                                        |
| 192.168.178.51                        | MacBook       |               |                                        |
| 192.168.178.58                        | Vacuum Robot  |               |                                        |
| 192.168.178.60                        | MacBook       |               |                                        |
| 192.168.178.62                        | iPhone        |               |                                        |
| 192.168.178.64                        | SmartTV       |               |                                        |
| 2003:c1:3720:f300:e96b:cc0b:be8f:e1e3 | GoogleHome    |               |                                        |
| fe80::d6f5:47ff:fe38:193b             | GoogleHome    |               |                                        |
| 2003:c1:3720:f300:99b8:9c2:5bc1:84be  | GoogleHome    |               |                                        |
| 192.168.0.1                           |               |               | Router|
| 192.168.0.2                           |               |               | Wifi Smart Plug                        |
| 192.168.0.8                           |               |               | Amazon Fire TV                         |
| 192.168.0.9                           |               |               | Sonos Wifi Loudspeaker                 |
| 192.168.0.14                          |               |               | iPad                                   |
| 192.168.0.22                          |               |               | ESP32 Microcontroller                  |
| 192.168.0.88                          |               |               | Raspberry Pi used for network sniffing |
| 192.168.0.121                         |               |               | ESP32 with Feinstaubsensor firmware    |


Start of the program

In [ ]:
# Program, run imports
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import time
import seaborn as sns
import requests
sns.set(style="darkgrid")

# used to skip merging and filtering
new_read = False

Create Pandas Dataframe

In [ ]:
if new_read:
    df_fabi = pd.read_csv("dumps/allPorts_fabian.csv", encoding = "latin")
    df_maxi = pd.read_csv("dumps/allPorts_maxi_v2.csv", encoding = "latin")
    df_sven = pd.read_csv("dumps/advanced-dumps-sven.csv", encoding = "latin")
    df = pd.concat([df_fabi, df_maxi, df_sven])
    print(df)

Filter Local Traffic

Print total number of captured packages

In [ ]:
if new_read:
    # source and destination should not start with 192.168 to filter local network. DNS should not be filtered.
    filtered = df.loc[~df['Source'].str.startswith("192.168", na=False) & df['Destination'].str.startswith("192.168", na=False) |
                    df['Source'].str.startswith("192.168", na=False) & ~df['Destination'].str.startswith("192.168", na=False) |
                    df['Protocol'].isin(['DNS'])]

    filtered = filtered.loc[~filtered['Protocol'].isin(['DHCP', 'ARP', 'MDNS', 'LLDP', 'SSDP', 'IGMP', 'IGMPv2', 'IGMPv3', 'ICMP', 'ICMPv4', 'ICMPv6', 'ieee1905', 'LLMNR'])]
    filtered = filtered.loc[~filtered['Destination'].isin(['255.255.255.255'])]


    df = filtered 

    # sort array by timestamp
    df = df.sort_values(by=['Time'])

    # save to csv
    df.to_csv('df.csv')
    # print (df.loc[df['Destination'].str.contains("255.255.255.255")])
else: 
    df = pd.read_csv('df.csv')
print(df)

print(len(df))

In [ ]:
# helper functions
def utcEntryToTimestamp(entry):
    if '.' in entry:
        row_entry = entry.split(".")[0]
    else:
        row_entry = entry.split(",")[0]
    TIME_FORMAT='%Y-%m-%d %H:%M:%S'
    ts = int(datetime.strptime(row_entry, TIME_FORMAT).timestamp())
    return ts

def utcRowToTimestamp(row):
    return utcEntryToTimestamp(row.at['Time'])
utcRowToTimestamp(df.iloc[0])
# print(df.loc[0].at['Time'])

## Protocols
We would like to start our investigations by getting a better insight about the Packet Types were are sending to the web.



 ### Packet Distribution

In [ ]:
# rank by most used protocols
df_ranked_protocols = df.groupby('Protocol').size()
print(df_ranked_protocols.nlargest(15))

# plot pie diagram
fig, ax = plt.subplots()
plt.title('Protocol Distribution by Amount of Packets')
ax.pie(df_ranked_protocols, labels=df_ranked_protocols.keys(), autopct='%1.1f%%',)
fig.show()

## Amount of Data Traffic per Protocol

In [ ]:
df_data_per_protocol = df.groupby('Protocol')['Length'].sum()
print(df_data_per_protocol.nlargest(15))

fig, ax = plt.subplots()
plt.title('Protocol Distribution by Data Traffic')
ax.pie(df_data_per_protocol, labels=df_data_per_protocol.keys(), autopct='%1.1f%%',)
fig.show()

### Average Data length per Protocol Type ###

In [ ]:
df_mean_protocol_packet_length = df.groupby('Protocol')['Length'].mean()
print(df_mean_protocol_packet_length.nlargest(15))

# plot
fig, ax = plt.subplots()
ax.tick_params(axis='x', which='major', labelsize=12)
ax.tick_params(axis='x', which='minor', labelsize=12)
plt.xlabel('Protocol')
plt.ylabel('Data [Byte]')
plt.title('Mean Data Length per Protocol Type')
ax.bar(df_mean_protocol_packet_length.keys(), df_mean_protocol_packet_length, align='center',)
plt.xticks(rotation=60, ha="right")
fig.show()

## Source addresses

In [ ]:
df_ranked_sources = df.groupby('Source').size()
print(df_ranked_sources.nlargest(25))

Local devices with most traffic sent

In [ ]:
print(df.loc[df['Source'].str.startswith("192.168")].groupby('Source').size())

## Destination Adresses

In [ ]:
df_ranked_destinations = df.groupby('Destination').size()
print(df_ranked_destinations.nlargest(25))

In [ ]:
# write ip address destinations to file
unique_dests = df['Destination'].unique()
file1 = open("destinations.txt","w")
for row in unique_dests:
    file1.write(row + "\n")
file1.close()

Local devices with most traffic received

In [ ]:
print(df.loc[df['Destination'].str.startswith("192.168")].groupby('Destination').size())

### IP Locations World Wide Sources

In [ ]:
rows = []
for address, count in df_ranked_sources.nlargest(30).iteritems(): # 30 entries limit 
    if address.startswith("192.168."):
        continue
    headers = { 'User-Agent': "keycdn-tools:https://www.example.com" }
    url = "https://tools.keycdn.com/geo.json?host={}".format(address)
    json_response = requests.get(url, headers=headers).json()
    #print(json_response)
    geo = json_response['data']['geo']
    #print(json_response)
    #print(geo)
    rows.append([geo['ip'], geo['longitude'], geo['latitude'], geo['isp']])
    
# as dataframe
df_coord = pd.DataFrame(rows, columns=["ip", "lng", "lat", "isp"])
print(df_coord)
df_coord.to_csv('ipLocations.csv')

# plot on world
g_world = gpd.GeoDataFrame(df_coord, geometry=gpd.points_from_xy(df_coord.lng, df_coord.lat))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black')
g_world.plot(ax=base, marker='o', color='red', markersize=5)
plt.show()

### IP Locations World Wide Destinations

In [ ]:
rows = []
for address, count in df_ranked_destinations.nlargest(30).iteritems(): # 30 entries limit 
    if address.startswith("192.168."):
        continue
    headers = { 'User-Agent': "keycdn-tools:https://www.example.com" }
    url = "https://tools.keycdn.com/geo.json?host={}".format(address)
    json_response = requests.get(url, headers=headers).json()
    #print(json_response)
    geo = json_response['data']['geo']
    #print(json_response)
    #print(geo)
    rows.append([geo['ip'], geo['longitude'], geo['latitude'], geo['isp']])
    
# as dataframe
df_coord = pd.DataFrame(rows, columns=["ip", "lng", "lat", "isp"])
print(df_coord)
df_coord.to_csv('ipLocations.csv')

# plot on world
g_world = gpd.GeoDataFrame(df_coord, geometry=gpd.points_from_xy(df_coord.lng, df_coord.lat))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black')
g_world.plot(ax=base, marker='o', color='red', markersize=5)
plt.show()

# Encryption #

To look at the percentage of encrypted traffic we will base our analysis solely on the used protocols. The reason behind this is that we were not able to capture the content of the traffic over time due to the memory intensity and therefore cannot look if the content is encrypted or not.

To distinguish between packages with encrypt content and packages with non-encrypt content we created the following list of protocols that are used to encrypt traffic:
- TLS
- SSH
- SSL
- IPsec
- PGP
- S/MIME
- UDPENCAP
- Kerberos
- ISAKMP
- WireGuard
- DTLSv1.2
- GQUIC
- QUIC
- HART_IP
- LWAPP

Based on this we get the following encryption distribution:

In [ ]:
encrypted_protocols = ['SSH', 'SSHv2', 'SSL', 'SSLv3', 'SSLv2', 'TLS', 'TLSv1', 'TLSv1.2', 'TLSv1.3', 'IPsec', 'ESP', 'PGP', 'S/MIME', 'Kerberos', 'ISAKMP', 'UDPENCAP', 'WireGuard', 'DTLSv1.2', 'GQUIC', 'HART_IP', 'LWAPP', 'QUIC']
encrypted_df = df.loc[df['Protocol'].isin(encrypted_protocols)]

content_length_encrypted = encrypted_df['Length'].sum()
content_length_unencrypted = df['Length'].sum()

labels = 'Encrypted', 'Non-Encrypted'
fig, ax = plt.subplots()
plt.title('Encryption Distribution')
ax.pie([content_length_encrypted, content_length_unencrypted - content_length_encrypted], explode=(0.1, 0), labels=labels, autopct='%1.1f%%',)
fig.show()

By inspecting the plot it can quickly be seen that the encrypted packages only take up a small portion of the traffic. However, this number can not be taken as an exact value but as an lower bound since packages send with the UDP and TCP protocol take up most of the traffic but could be encrypted as well. Since don't have the content of the packages we cannot further investigate into the encryption of these. But we suspect that the ecryption percentage would be a lot higher.

# IPv6 vs IPv4

In [ ]:
ether_distribution = df.groupby('EtherType').size()

fig, ax = plt.subplots()
plt.title('EtherType Distribution')
ax.pie(ether_distribution, labels=ether_distribution.keys(), autopct='%1.1f%%',)
fig.show()

## DNS 

We would like to investigate further our collected data. Now, we are particular interested in DNS requests. therefore, we filter the data by the protocol "DNS".

In [ ]:
df_dns = df[df['Protocol'].isin(['DNS'])]
print(df_dns.head(1))

### Used DNS Server

Various DNS Resolvers exist on the Internet. We would like to find out which our devices are using during their operation. 
Some devices can be configured to use specif DNS Servers, some a DNS resolver hardcoded in their firmware. In man cases the resolve request is just forwarded to the router, who takes care of this. 
By grouping our DNS Destinations and counting the requests, we see which resolvers were primarily used.

In [ ]:
df_dns_server = df_dns.loc[~df_dns['Info'].str.contains("response")]
print(df_dns_server.groupby(['Destination']).size())

We see that most of our DNS requests were send to our routers or well known DNS resolvers, such as Google's 8.8.8.8. However, we also found tree unknown DNS resolvers. A print out of these requests reveals that the Feinstaubsensor IoT device is talking to changing DNS Servers to resolve its destinations.
The other one seems to be an IPv6 DNS resolver, that is used by our Amazon fire TV to resolve its requests. 

In [ ]:
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("195.234.128.139")].head(5))
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("217.68.162.126")].head(5))
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("2a02:2457:10c:101::126")].head(5))

Another great insight is figuring out what URLs are actually being resolved. This can also be used to track a devices browsing behavior. We count the requests by URL to get a better overview. \
An interesting finding is that many of our top resolved URLs are requested by static devices, such as the Feinstaubsensor, Google Home or the Amazon Fire TV stick. 

Especially on personal computers and mobile devices, a brosing behavior from DNS requests can be found. This includes our love for Reddit. 👨🏽‍🚀

In [ ]:
# only look at standart query dns requests
df_dns_requests = df_dns[df_dns['Info'].str.contains('Standard query')]
# exclude responses from count
df_dns_requests = df_dns_requests[~df_dns_requests['Info'].str.contains('response')]
# display URLS
df_dns_urls = df_dns_requests['Info'].apply(lambda x: x.split(' ')[-1]).value_counts()
print(df_dns_urls.head(30))

### Data traffic over time ###

In [ ]:
def timeMapping(x):
    # include time difference UTC+1
    time = datetime.utcfromtimestamp(x*min_15_duration + 3600)
    if time.minute == 0 and time.hour % 3 == 0:
        return time.strftime("%H:%M")
    else:
        return ""

# find first and last timestamp, then create data structure
min_15_duration = 60 * 15
# df_time_mod = df.copy()
print('Compute intensive task 1/3...')
df['index-time'] = df['Time'] # .apply(lambda x: x)) # map to 15 min window
df['index-time'] = df['index-time'].apply(lambda x: utcEntryToTimestamp(x))

first_entry = df.iloc[0].at['index-time'] // min_15_duration
last_entry = df.iloc[-1].at['index-time'] // min_15_duration

print('from ', df.iloc[0].at['Time'], " to ", df.iloc[-1].at['Time'])

count_packets = np.zeros(last_entry - first_entry + 1)
length_packets = np.zeros(last_entry - first_entry + 1)

x_values_packets = list(range(first_entry, last_entry + 1))
print('Amount of Intervals',len(x_values_packets))

mapping_res = list(map(timeMapping, x_values_packets))

print('Compute intensive task 2/3...')
df['index-time'] = (df['index-time'] // min_15_duration) - first_entry


In [ ]:
count_packets = np.zeros(last_entry - first_entry + 1)
length_packets = np.zeros(last_entry - first_entry + 1)
print('Compute intensive task 3/3...')
time_start = time.time()
for row in df.to_dict('records'):
    count_packets[row['index-time']] += 1
    length_packets[row['index-time']] += row['Length']

# old solution:
#for index, row in df.iterrows():
#    count_packets[row.at['index-time']] += 1
#    length_packets[row.at['index-time']] += row.at['Length']

time_end = time.time()
print(time_end - time_start)

length_packets = length_packets // 1000
#print(count_packets)
#print(length_packets)

fig, ax = plt.subplots()
plt.xlabel('Time')
plt.ylabel('Packets')
plt.title('Packets per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, count_packets, color='black')
fig.show()

fig, ax = plt.subplots()
plt.xlabel('Time')
plt.ylabel('Data [kB]')
plt.title('Data traffic per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, length_packets, color='black')
fig.show()

End of the notebook